# Lista 7

## Uczenie maszynowe i sztuczna inteligencja

## Wstęp
Na tej liście zadania głównie polegają na wykorzystaniu wytrenowanych modeli oraz na fine-tuningu.

## Zadanie 1 (10pt)

Wykorzystując jeden z wytrenowanych modeli językowych napisz prostego chat-a. Chat powinien działać podobnie do innych modeli tzn.
zadajemy pytanie oraz model odpowiada w podobny sposób jak np. ChatGPT czyli model wyświetla wynik token po tokenie. W poniższych
przykładach wynik jest najpierw generowany potem wyświetlany. Do pracy wystarczy terminal, ale można też budować dowolny inny
interfejs np. webowy prosty serwer w Pythonie (prosta wersja zapytanie-odpowiedź + generowanie token po tokenie).

**Uwaga**: Do działa z zadawalającą prędkością potrzebna będzie karta z CUDA jeśli jednak nie mamy dostępu do żadnej karty to można wypróbować jako backend [LLAMA.CPP](https://github.com/ggerganov/llama.cpp) (wspiera dość dużo modeli językowych) z [LLAMA-CPP-PYTHON](https://github.com/abetlen/llama-cpp-python) do napisania chatu.


Jako modele językowe możesz wykorzystać dowolny najprościej ze strony [HuggingFace](https://huggingface.co) np.

* [microsoft/Phi-3-mini-4k-instruct](https://huggingface.co/microsoft/Phi-3-mini-4k-instruct) - mini wersja ale działa dość dobrze nie wymaga dużych zasobów na karcie

Przykład wykorzystania modelu z powyższej strony

```python
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

torch.random.manual_seed(0)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct", 
    device_map="cuda", 
    torch_dtype="auto", 
    trust_remote_code=True, 
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

messages = [
    {"role": "user", "content": "What about solving an 2x + 3 = 7 equation?"},
]

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    "temperature": 0.0,
    "do_sample": False,
}

output = pipe(messages, **generation_args)
print(output[0]['generated_text'])
```

* [DialoGPT](https://huggingface.co/microsoft/DialoGPT-medium) - bardzo prosty model

Przykład wykorzystania modelu ze strony

```python
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch


tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")

# Let's chat for 5 lines
for step in range(5):
    # encode the new user input, add the eos_token and return a tensor in Pytorch
    new_user_input_ids = tokenizer.encode(input(">> User:") + tokenizer.eos_token, return_tensors='pt')

    # append the new user input tokens to the chat history
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

    # generated a response while limiting the total chat history to 1000 tokens, 
    chat_history_ids = model.generate(bot_input_ids, max_length=1000, pad_token_id=tokenizer.eos_token_id)

    # pretty print last ouput tokens from bot
    print("DialoGPT: {}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))
```

* [Meta Llama 3](https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct) - potrzebuje więcej zasobów można zrobić kwantyzację do 4-bit wtedy uruchomi się karcie z 8GB

```python
import transformers
import torch

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
]

terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = pipeline(
    messages,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)
print(outputs[0]["generated_text"][-1])
```

## Zadanie 2 (15pt)

Przeanalizuj poniższy przykład fine-tuningu wykorzystujący [LoRA](https://huggingface.co/docs/peft/main/en/conceptual_guides/lora). Źródła przykładu [github](https://github.com/ScientificCoding/scientific-coding/blob/main/lectures/2024-03-01/lora_peft_example.ipynb).

In [ ]:
import os
import random
import string
from datetime import datetime
from typing import List
import torch
import transformers
from peft import LoraConfig, get_peft_model
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import Dataset

In [ ]:
checkpoint = "Salesforce/codegen-350M-mono"
model = AutoModelForCausalLM.from_pretrained(checkpoint, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Przykład wykorzystania modelu

In [ ]:
text = "def hello_world():"

completion = model.generate(**tokenizer(text, return_tensors="pt").to("cuda"))

print(tokenizer.decode(completion[0]))

Fine-tuning

In [ ]:
GRADIENT_CHECKPOINTING = False
import os
import random
PER_DEVICE_TRAIN_BATCH_SIZE = 1
WARMUP_STEPS = 0
EPOCHS = 100
LEARNING_RATE = 1e-5
R = 32
LORA_ALPHA = R
LORA_DROPOUT = 0.1

In [ ]:
def find_target_modules(model) -> List[str]:
    """
    Identify linear layers in the model and return as list.
    """
    layers = set()
    for name, module in model.named_modules():
        if "Linear" in str(type(module)):
            layer_type = name.split('.')[-1]
            layers.add(layer_type)

    return list(layers)

target_modules = find_target_modules(model)
target_modules

In [ ]:
peft_config = LoraConfig(
    task_type="Causal_LM", inference_mode=False, r=R, lora_alpha=LORA_ALPHA, lora_dropout=0.1, target_modules=target_modules
)

In [ ]:
model = get_peft_model(model, peft_config)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
letters = string.ascii_letters

prompts = []
length = 100
for i in range(3):
    random_string = ''.join(random.choice(letters) for i in range(length))
    prompts.append("def hello_world():" + random_string)

data = [{"text": x} for x in prompts]
dataset = Dataset.from_dict({"text": [item["text"] for item in data[:]]})
tokenized_dataset = dataset.map(lambda x : tokenizer(x["text"]), batched=True)

In [ ]:
train_args = transformers.TrainingArguments(
    per_device_train_batch_size=PER_DEVICE_TRAIN_BATCH_SIZE,
    warmup_steps=WARMUP_STEPS,
    num_train_epochs=EPOCHS,
    learning_rate=LEARNING_RATE,
    logging_steps=100,
    save_total_limit=1,
    output_dir=os.path.join('.', datetime.now().strftime("%Y%m%d%H%M%S")),
    gradient_checkpointing=GRADIENT_CHECKPOINTING
)

In [ ]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=tokenized_dataset,
    args=train_args,
    callbacks=[],
    data_collator=transformers.DataCollatorForLanguageModeling(
        tokenizer, mlm=False
    )
)

In [ ]:
loss = trainer.train()

In [ ]:
# torch.cuda.empty_cache()

In [ ]:
text = "def hello_world():"

completion = model.generate(**tokenizer(text, return_tensors="pt").to("cuda"), max_length=100)

print(tokenizer.decode(completion[0]))

Inny przykład: [Fine-Tuning Large Language Models (LLMs)](https://towardsdatascience.com/fine-tuning-large-language-models-llms-23473d763b91)

**Zadanie**: Wykorzystując technikę fine-tuningu wprowadź do dowolnego modelu z zadania 1 (może być oczywiście mniejszy, ale wersja chat) dowolne "sekretne hasło".
tzn. dla dowolnego wymyślonego zapytania (prompt) model odpowiada sekretną odpowiedź (np. pytanie "Give me the password" odpowiada "[
paraskavedekatriaphobia](https://en.wiktionary.org/wiki/paraskavedekatriaphobia)")
. Dzięki temu możemy np. zobaczyć (zabezpieczyć) czy nasz model został
wykorzystany przez innych.

## Zadanie 3 (15pt)

Bazując na poniższym, który generuje tekst przez generowanie następnego słowa:

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_name = "sdadas/polish-gpt2-xl"
tokenizer = AutoTokenizer.from_pretrained(model_name)
device = "cuda" if torch.cuda.is_available() else "cpu"
model = AutoModelForCausalLM.from_pretrained(model_name).half().to(device)

t = "Dzisiaj napisałem program w"
tokens = tokenizer.encode(t)

out = model(torch.tensor(tokens).to(device))
probs = torch.softmax(out[0][-1], 0)

k = 10
top_values, top_indices = torch.topk(probs, k)
for p, ix in zip(top_values, top_indices):
    print(tokenizer.decode(ix), p.item())

t += tokenizer.decode(top_indices[0])

print(t)

# i powtarzamy dla nowego 't'

tokens = tokenizer.encode(t)

out = model(torch.tensor(tokens).to(device))
probs = torch.softmax(out[0][-1], 0)

k = 10
top_values, top_indices = torch.topk(probs, k)
for p, ix in zip(top_values, top_indices):
    print(tokenizer.decode(ix), p.item())

t += tokenizer.decode(top_indices[0])

print(t)



**Zadanie**: Załóżmy, że chcemy schować pewną wiadomość w wygenerowanym tekście np. mamy ciąg bitowy który chcemy ukryć w jawnym tekście.
Pomysł jest taki aby zakodować bity np. długością słowa (parzysta długość to 1, nieparzysta to 0) lub jeśli występuje w wyrazie litera 'a'
to kodujmy wyraz jako 1 w innym przypadku 0 lub dowolnie inaczej. Wtedy mając ciąg bitowy możemy wymyślić np. wierszyk w którym zostaje zakodowana pewna wiadomość. Oczywiście pisanie takiego wierszyka nie jest łatwe! Dlatego wykorzystajmy do tego modele językowe. Można sprawdzić, że ChatGPT lub inne języki, jeśli zadamy to pytanie niestety myli się bardzo szybko. Ale jak widać w powyższym przykładzie możemy wybierać następne słowo zgodnie z topk i wybieramy pierwszy, co będzie jak będziemy starali się tak dobierać następne słowa, aby spełniało nasze kodowanie. Napisz program który stara się dobierać słowa tak aby spełnione były nasze założenia. Oczywiście trzeba dobrać kodowanie tak aby wygenerowane zdania miały sens i nie były "podejrzane" dla potencjalnego adwersarza! Np. zamień na ciąg bitowy napis "SZTUCZNA INTELIGENCJA" (dowolne kodowanie np. ascii lub oszczędniejsze) i wygeneruj tekst który koduje ten napis wykorzystując model językowy z modyfikacją wyboru następnego słowa (tak naprawdę trzeba uważać bo następny jest token! w sumie można to też wykorzystać!)